# Recursive Neural Network for Sentiment Analysis

## Parse Tree for Sentiment Analysis

* Two ways so far of representing sentences:
    1. bag of words
    2. sequence of words


* The third way would be: parse trees (with more structural information)

[example]

* Parse Trees
    * Relations
        * e.g., left child and right child
    * A node can have any number of children
        * We will use binary tree such that a node either has no child or it has two children
    * With recurrent NNs, we did next word prediction $P(x(t)| x(t-1),...,x(0))$. In parse trees, there is no such thing as "next" since it is a hierarchical structure and not a sequence
        * We will transform parse trees to sequences such that we can use recurrent neural network to train these parse trees. 

* Sentiment analysis
    * bag-of-words representing a sentence can not handle negation.
    * Recursive neural nets solve this problem
        * Best prior to recursive neural nets: ~80%
        * Recursive neural nets can achieve 85% - 90%
    * Why recursive neural nets work on sentiment analysis
        * Parse tree is structured to identify dependencies between words. 
            * If you have negation, then it is very easy to reverse whatever is the phrase that it is negating
        * The neural network can use them to do negation
        * We need to label every node in the tree
            * labeling sentences with parse tree structure is labor-tensive. This is one of the reasons why progress of adopting recurive neural network is hard compared to other neural network architecture. 
        
            
        

* Parse Tree Structure in Sentiment analysis
    * It has basic structure of a parse tree as we introduced above
    * only leaves of the tree represent words
    * Any inner nodes represents phrase made up of words


* Data 
    * Get data from [Stanford Sentiment Analysis](http://nlp.stanford.edu/sentiment)
    * Look for file: <b>trainDevTestTrees_PTB.zip</b>
    * Each node is labeled using Amazon Mechanicl Turk, and have rating: 1-5, in which 3 is neural, 1 is the most negative and 5 is the most positive
    * A sentence has the format of: (5 (5 great) (3 movie))
    * Represent it in tree structure:
    
    [example]
    
    * We need to transform each sentence into a sequence that embodies the parse tree structure

## Recursive Neural Network for Parse Tree

* Recursiveness: A node's value is defined by its children, and in turn children's values are defined by their children respectively, and so on so forth...
* We need weights (i.e., weight matrix) one each edge between a parent node and one of its child to tell how this child connects to the parent.
    * Since we use binary tree, we only neet two weights for each node (i.e., inner node): one for left child and one for right child
    > Note that although we may have multiple nodes that each has its own children, we use the same left weight matrix for all nodes to connect their left child, and the same right matrix for all nodes to connect their right child
    
    [picture]
    
    $$ x_1 = We_{1} = \text{word embeddings for word 1} $$
    
    $$ x_2 = We_{2} = \text{word embeddings for word 2} $$
    
    $$ h_1 = f(W_{left} x_1 + W_{right} x_2 + b) $$
    
    $$ h_{root} = f(W_{left} h_1 + W_{right} x_3 + b) $$
    
where $f()$ can be any of the activation functions: relu, tanh and sigmoid.

* What is the dimensionality of the weights?
    * Since we use the same set of left and right weight matrix for all inner nodes, the shape of the weigth matrix shoule be $(D, D)$, where $D$ is the pre-defined dimension.
    * The bias $b$ should be a $D$ dimention vector.
  
* How to get the output?
* Any node can be labeled (we can choose whether or not to use inner nodes for training)

$$ p(y | h) = softmax(W_o h + b_o) $$

where $h$ can be root, inner node or leave node (i.e., word); $W_o$ is the weight matrix for the output and $b_o$ is the bias for the output.

[picture]

**N-ary Trees**

* Let us first define value for a node of three children:

$$ h = f(W_{1} x_1 + W_{2} x_2 + W_{3} x_2 + b) $$

* We can extend this to general case:

$$ h = f(\sum_{i} W_{rel(h, i)} x_{i} + b) $$

* We can define the weight matrix W as a matrix with shape $(R, D, D)$, where $R$ is the dimension for all possible parent-child relationships.
* function $rel(h, i)$ returns a number form 1 to R telling us the type of relationship between parent $h$ and child $i$.
* You can think of binary tree as $R=2$

## Recursive NNs to Recurrent NNs


* We know that Recursive NNs takes trees as input, while Recurrent NNs takes sequences as input
    * We convert trees to sequences
* How to do the conversion?
* Arrange the parse tree in a list such that children always come before parents
    * We can do this using post-order traversal algorithm
    * While we are traversing the parse tree in post order, we create three arrays/lists: parent array, relation array and word array.
        * parent array: index represents nodes' ID, value represents index of parent. If a node has no parent, the value would be -1
        * relation array: index represents nodes' ID, value represents index of relation to parent. If a node has no parent, the value would be -1
        * word array:  index represents nodes' ID, value represents index of word. If a node has no word, the value would be -1

[three or more pictures]

* Summary
    * Three separate arrays to store a tree
    * parents/relations/words
    * Using post-order traversal to create the three arrays

## How to write code in Theano

**Theano scan**

```python
# initialize the hidden matrix that each row represents a node in the original parse tree
hiddens = T.zeros((words.shape[0], D)) 
h, _ = theano.scan(
    fn=recurrence,
    squences=range(n)
    outputs_info=[hiddens],
    non_sequences=[parents, relations, words]
)
``` 

**Recurrence**

* Two things we want to do

(1) calculate value at hidden node, n

```python
if words[n] >= 0:
    hiddens[n] = We[words[n]]
else:
    hiddens = T.set_subtensor(hiddens[n], f(hiddens[n] + bh))
```

> Note that hiddens[n] already contains values from its children: $W_{left} x_1 + W_{right} x_2$. This is because we add $W_{left} x_1$ to hidden[n] when we are processing left child of node n and add $ W_{right} x_2$ to hidden[n] when we are processing right child of node n, and both left and right children are processd before the processing of node n.

(2) update parent if exists

```python
p = parents[n]
r = relation[n]
if p > 0:
    hiddens = T.set_subtensor(hiddens[p], hiddens[p] + hiddens[n].dot(Wh[r]))
```

> If current node n has a parent, this node can be either left child or right child, and the relation is defined by $r = relation[n]$. 

> $Wh$ is a $(R, D, D)$ matrix, of which the first dimension is indexes of relation. For binary three, 0 represents left child while 1 represents right child.


** Use if statement in Theano **

* Theano must create a graph and thus it can not contains arbitrary Python code.
    * e.g., we use theano.scan() to perform the functionality of for loop
* In theano we use <b style="color:red">T.switch</b> to perform functionality of if statement
* The API for T.swich:

```
new_value = T.switch(condition, value if condition is true, value if condition is false)
```

* Use T.switch for (1):

```python
hiddens = T.switch(
    T.ge(words[n], 0),
    We[words[n]],
    T.set_subtensor(hiddens[n], f(hiddens[n] + hb),
)

```

### References:

* [Stanford Sentiment Analysis](https://nlp.stanford.edu/sentiment/)
* [Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)
* [Parsing With Compositional Vector Grammars](https://nlp.stanford.edu/pubs/SocherBauerManningNg_ACL2013.pdf)